. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

In [31]:
ratings = pd.read_csv('ratings.csv')

In [30]:
ratings.head()
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')
z = ratings['userId'].value_counts()
ratings['Count_userId'] = ratings['userId'].map(z)
ratings_over100 = ratings[(ratings['Count_userId'] >=100)]
results = ratings_over100.groupby('userId').agg({'date': ['min', 'max'] })
results['lifetime'] = results['date']['max'] - results['date']['min']
results['lifetime'].head(20)


userId
4       2 days 08:32:40
5       0 days 00:35:01
8       0 days 23:39:47
15   5455 days 22:38:16
17      0 days 02:14:13
19      0 days 01:28:02
21     15 days 19:17:12
22      1 days 01:24:51
23    212 days 07:02:09
26    234 days 13:47:41
30   1340 days 21:11:16
33     50 days 03:23:31
34      0 days 00:47:15
36     68 days 20:34:50
38    694 days 02:47:45
41    184 days 09:08:15
43      0 days 03:51:25
48   1577 days 17:32:57
56     99 days 02:55:20
57      0 days 01:12:03
Name: lifetime, dtype: timedelta64[ns]

**Задание  2**
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [ ]:
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [27]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd


,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [22]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [24]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [25]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [39]:
clients_revenue = (
    client_base
    .merge(auto, how='outer')
    .merge(air, how='outer')
    .merge(rzd, how='outer')
)
clients_revenue_1 = clients_revenue[['client_id','auto_revenue','air_revenue','rzd_revenue']].fillna(0)
clients_revenue_1

,client_id,auto_revenue,air_revenue,rzd_revenue
0,111,0.0,0.0,1093.0
1,112,0.0,0.0,2810.0
2,113,57483.0,0.0,10283.0
3,114,83.0,0.0,5774.0
4,115,912.0,81.0,981.0
5,116,4834.0,4.0,0.0
6,117,98.0,13.0,0.0
7,118,0.0,173.0,0.0


In [41]:
clients_revenue.fillna(0)

,client_id,address,auto_revenue,air_revenue,rzd_revenue
0,111,Комсомольская 4,0.0,0.0,1093.0
1,112,Энтузиастов 8а,0.0,0.0,2810.0
2,113,Левобережная 1а,57483.0,0.0,10283.0
3,114,Мира 14,83.0,0.0,5774.0
4,115,ЗЖБИиДК 1,912.0,81.0,981.0
5,116,Строителей 18,4834.0,4.0,0.0
6,117,Панфиловская 33,98.0,13.0,0.0
7,118,Мастеркова 4,0.0,173.0,0.0


**Задание 3**
1. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
2. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
3. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
4. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
CSV 

Не очень понятно, как связаны координаты с визитами и покупками. Если никак, то полагаю, что их можно связать по максимальной приближенности координатов пользователя к координатам места визита. Если покупа совершена онлайн, то смысла в том нет.

4. Из яндекса или гугла можно выгрузить точный адрес по координатам. Предположить, где клиент живет или работает. Если например покупка совершена в 10 утра в воскресенье и координаты совпали, то где-то здесь он живет, если в четвер в 6 вечера, то вероятнее всего работает.
Можно строить ему маршруты в приложении от его местоположения до магазина, предлагать магазины рядом и тд